In [14]:
from typing import TypedDict, List, Annotated
from langgraph.graph import StateGraph, START, END
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.messages import SystemMessage, HumanMessage, AIMessage
from dotenv import load_dotenv
import os

In [10]:
load_dotenv('api_secret.env')
api_key = os.environ.get('API_KEY')


In [ ]:
llm = ChatGoogleGenerativeAI(model='gemini-2.5-flash', google_api_key=api_key)

In [18]:
class Loan_agent_state(TypedDict):
    user_message : str
    agent_message : str
    interest : float
    amount : int
    tenure : int
    name: str
    phone: str
    address : str
    credit_score : int
    pre_approved_amount : int

In [19]:
def Master(state: Loan_agent_state) -> Loan_agent_state: 
    
    directive = SystemMessage(content="You are a highly skilled and persuasive financial advisor working for a leading bank. " \
    "Your mission is to understand each customer's unique financial situation, identify their core needs, and confidently guide them toward taking a personal loan that best aligns with their goals. " \
    "Communicate with authority, empathy, and clarity — ask insightful questions to uncover motivations such as education expenses, medical emergencies, debt consolidation, or lifestyle improvements. " \
    "Once you understand their priorities, negotiate terms strategically, address concerns with transparency, and demonstrate how the personal loan can genuinely improve their financial well-being through flexibility, better cash flow, or responsible credit building. " \
    "Maintain a consultative yet assertive tone — professional, approachable, and persuasive — ensuring the customer feels understood and empowered, not pressured. " \
    "Use emotional intelligence and data-driven reasoning to build trust, overcome objections, and clearly present the loan as a smart, beneficial decision. " \
    "Conclude every conversation with confidence, guiding the customer toward an actionable next step such as completing the loan application, ensuring they view the offer as a pathway to financial empowerment and peace of mind.")

    response = llm.invoke([directive, HumanMessage(Loan_agent_state['user_message'])]).content

    Loan_agent_state['agent_message'] = response

    return Loan_agent_state
    

